# MIHAC — Reporte de Validación Empírica

**Notebook**: `02_validation_report.ipynb`  
**Versión**: 1.0  
**Motor**: MIHAC v1.0 — Motor de Inferencia Heurística para Aprobación de Créditos  
**Dataset**: German Credit (UCI, 1,000 registros)  
**Fecha**: Febrero 2026  

---

## Objetivo

Documentar la **validación empírica** del motor MIHAC mediante backtesting contra el  
German Credit Dataset. Este notebook constituye la evidencia cuantitativa para el  
capítulo de resultados de la tesis.

## Contenido

1. Configuración e Imports  
2. Carga de Datos  
3. Backtesting (1,000 registros)  
4. Métricas de Desempeño  
5. Visualizaciones  
6. Análisis de Errores (FP / FN)  
7. Calibración de Pesos  
8. Comparación Antes vs Después  
9. Discusión y Limitaciones  
10. Conclusiones

In [1]:
# ═══════════════════════════════════════════════════════════
# 1. CONFIGURACIÓN E IMPORTS
# ═══════════════════════════════════════════════════════════

import sys
import warnings
import time
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Suprimir warnings no críticos
warnings.filterwarnings('ignore')

# Configuración visual
plt.rcParams.update({
    'figure.figsize': (12, 6),
    'figure.dpi': 100,
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
})
sns.set_style('whitegrid')

# Paleta MIHAC
VERDE  = "#2ECC71"
ROJO   = "#E74C3C"
AZUL   = "#2E74B5"
AMBAR  = "#F39C12"
GRIS   = "#95A5A6"

# Path del proyecto
PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == 'notebooks':
    PROJECT_ROOT = PROJECT_ROOT.parent
sys.path.insert(0, str(PROJECT_ROOT))

# Imports MIHAC
from core.engine import InferenceEngine
from data.mapper import GermanCreditMapper
from validation.metrics import MIHACMetrics, MIHACPlots
from validation.backtesting import Backtester
from validation.calibrator import WeightCalibrator

print(f"Raíz del proyecto: {PROJECT_ROOT}")
print(f"Python: {sys.version}")
print("✓ Todos los módulos importados correctamente")

Raíz del proyecto: c:\Users\marco\OneDrive\Desktop\Sistema Experto\mihac
Python: 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]
✓ Todos los módulos importados correctamente


---
## 2. Carga de Datos

Se utiliza `GermanCreditMapper` para descargar y transformar el German Credit Dataset  
del repositorio UCI. El mapper convierte las 20 variables originales del dataset  
(codificadas en formato categórico/numérico alemán de 1994) a las **9 variables de  
entrada del motor MIHAC**, más la etiqueta real (`etiqueta_binaria`).

**Convención de etiquetas:**
- `etiqueta_binaria = 1` → Buen pagador (clase 1 en German Credit)
- `etiqueta_binaria = 0` → Mal pagador (clase 2 en German Credit)

In [2]:
# ═══════════════════════════════════════════════════════════
# 2. CARGA DE DATOS
# ═══════════════════════════════════════════════════════════

mapper = GermanCreditMapper()
df = mapper.load_and_transform(None)  # Descarga de UCI

n_total = len(df)
n_buenos = int((df["etiqueta_binaria"] == 1).sum())
n_malos = n_total - n_buenos

print(f"{'='*50}")
print(f"German Credit Dataset — Resumen")
print(f"{'='*50}")
print(f"Total registros:     {n_total}")
print(f"Buenos pagadores:    {n_buenos} ({n_buenos/n_total*100:.1f}%)")
print(f"Malos pagadores:     {n_malos} ({n_malos/n_total*100:.1f}%)")
print(f"\nVariables MIHAC:     {list(df.columns[:9])}")
print(f"Etiqueta:            etiqueta_binaria")
print(f"\nPrimeros 5 registros:")
df.head()

German Credit Dataset — Resumen
Total registros:     1000
Buenos pagadores:    700 (70.0%)
Malos pagadores:     300 (30.0%)

Variables MIHAC:     ['edad', 'ingreso_mensual', 'total_deuda_actual', 'historial_crediticio', 'antiguedad_laboral', 'numero_dependientes', 'tipo_vivienda', 'proposito_credito', 'monto_credito']
Etiqueta:            etiqueta_binaria

Primeros 5 registros:


,edad,ingreso_mensual,total_deuda_actual,historial_crediticio,antiguedad_laboral,numero_dependientes,tipo_vivienda,proposito_credito,monto_credito,etiqueta_real,etiqueta_binaria
0,67,8183.00,2045.75,0,8,1,Propia,Consumo,12274.5,1,1
1,22,10848.18,3796.86,2,2,1,Propia,Consumo,50000.0,2,0
2,49,15283.33,5349.17,0,5,1,Propia,Educacion,22008.0,1,1
3,45,16420.83,5747.29,1,5,1,Familiar,Consumo,50000.0,1,1
4,53,11836.81,4142.88,0,2,1,Familiar,Consumo,50000.0,2,0


In [3]:
# Estadísticas descriptivas de las variables MIHAC
vars_mihac = [
    "edad", "ingreso_mensual", "total_deuda_actual",
    "antiguedad_laboral", "numero_dependientes", "monto_credito"
]
print("Estadísticas descriptivas de variables numéricas:\n")
df[vars_mihac].describe().round(2)

Estadísticas descriptivas de variables numéricas:



,edad,ingreso_mensual,total_deuda_actual,antiguedad_laboral,numero_dependientes,monto_credito
count,1000.00,1000.00,1000.00,1000.00,1000.0,1000.00
mean,35.55,18632.57,5698.88,3.57,1.0,27626.85
std,11.38,39150.26,13549.63,3.02,0.0,15070.97
min,19.00,3000.00,150.00,0.00,1.0,2625.00
25%,27.00,4200.00,1179.80,2.00,1.0,14337.75
50%,33.00,7495.64,2227.00,2.00,1.0,24354.75
75%,42.00,16775.39,4999.84,8.00,1.0,41708.62
max,75.00,651700.00,293265.00,8.00,1.0,50000.00


---
## 3. Backtesting (1,000 registros)

El **backtesting** ejecuta el motor MIHAC sobre los 1,000 registros del German Credit  
Dataset y compara los dictámenes automáticos contra las etiquetas reales.

**Pipeline de 7 pasos:**
1. Cargar datos transformados (mapper)
2. Evaluar cada registro con `InferenceEngine`
3. Construir vectores `y_real` / `y_pred` / `scores`
4. Calcular métricas con `MIHACMetrics`
5. Generar visualizaciones con `MIHACPlots`
6. Análisis de errores (FP y FN desagregados)
7. Generar reporte resumen

**Convención para métricas binarias:**  
`REVISION_MANUAL` se clasifica como **rechazo** (y_pred = 0). Justificación: si el  
motor no tiene confianza suficiente para aprobar autónomamente, el tratamiento  
conservador es contarlo como rechazo.

In [4]:
# ═══════════════════════════════════════════════════════════
# 3. BACKTESTING — Ejecución del Pipeline
# ═══════════════════════════════════════════════════════════

bt = Backtester()
reporte = bt.run(data_path=None, verbose=True)

MIHAC — Backtesting con German Credit Dataset

[Paso 1/7] Cargando datos...
  ✓ 1000 registros cargados
  Buenos pagadores: 700 (70.0%)
  Malos pagadores:  300 (30.0%)

[Paso 2/7] Evaluando 1000 registros...
  ... 200/1000 evaluados (1177 reg/seg)
  ... 400/1000 evaluados (1289 reg/seg)
  ... 600/1000 evaluados (1372 reg/seg)
  ... 800/1000 evaluados (1416 reg/seg)
  ... 1000/1000 evaluados (1430 reg/seg)
  ✓ 1000 evaluaciones completadas en 0.7s (1430 reg/seg)

[Paso 3/7] Construyendo vectores y_real / y_pred...
  ✓ Vectores construidos (n=1000)
  APROBADOS:        329
  RECHAZADOS:       546
  REVISION_MANUAL:  125 (→ contados como rechazo)

[Paso 4/7] Calculando métricas...
  Accuracy:    0.4590
  Precision:   0.7416
  Recall:      0.3486
  F1-Score:    0.4742
  Specificity: 0.7167
  AUC-ROC:     0.5512
  Costo Asim.: 0.7960

[Paso 5/7] Generando visualizaciones...
  ✓ Listas para guardar con save_report()

[Paso 6/7] Analizando errores (FP y FN)...
  Falsos Positivos (FP): 85
    P

In [5]:
# Resumen de distribución de dictámenes
dist = reporte["distribucion_dictamenes"]
print(f"\n{'='*50}")
print("Distribución de Dictámenes del Motor MIHAC")
print(f"{'='*50}")

# Tabla formateada
dictamen_df = pd.DataFrame([
    {"Dictamen": k, "Cantidad": v, "Porcentaje": f"{v/n_total*100:.1f}%"}
    for k, v in dist.items()
])
print(dictamen_df.to_string(index=False))

# Gráfico de barras de distribución
fig, ax = plt.subplots(figsize=(8, 5))
colores = [VERDE, ROJO, AMBAR]
bars = ax.bar(dist.keys(), dist.values(), color=colores, edgecolor="white", linewidth=1.5)

for bar, val in zip(bars, dist.values()):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 10,
            f"{val}\n({val/n_total*100:.1f}%)", ha="center", va="bottom",
            fontweight="bold", fontsize=12)

ax.set_title("Distribución de Dictámenes — MIHAC sobre German Credit", fontsize=14, fontweight="bold")
ax.set_ylabel("Cantidad de registros")
ax.set_ylim(0, max(dist.values()) * 1.25)
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
plt.tight_layout()
plt.show()


Distribución de Dictámenes del Motor MIHAC
       Dictamen  Cantidad Porcentaje
       APROBADO       329      32.9%
      RECHAZADO       546      54.6%
REVISION_MANUAL       125      12.5%


---
## 4. Métricas de Desempeño

Se calculan **6 métricas principales** para evaluar el motor:

| Métrica | Objetivo Tesis | Descripción |
|---------|---------------|-------------|
| Accuracy | > 0.70 | Proporción de clasificaciones correctas |
| Precision | > 0.75 | De los aprobados, ¿cuántos eran realmente buenos? |
| Recall (Sensibilidad) | > 0.65 | De los buenos pagadores, ¿cuántos fueron aprobados? |
| F1-Score | > 0.70 | Media armónica de Precision y Recall |
| AUC-ROC | > 0.70 | Capacidad discriminatoria general |
| Costo Asimétrico | < 0.25 | Ponderado 4:1 (FP:FN), penaliza aprobar morosos |

**Costo asimétrico** (ratio 4:1):  
- FP (aprobar moroso) → pérdida directa de capital, peso = 4  
- FN (rechazar solvente) → costo de oportunidad, peso = 1  
- Ratio estándar en literatura de scoring crediticio

In [7]:
# ═══════════════════════════════════════════════════════════
# 4. MÉTRICAS DE DESEMPEÑO
# ═══════════════════════════════════════════════════════════

metricas = reporte["metricas"]

# Tabla de métricas con objetivos
objetivos = {
    "accuracy": ("Accuracy", 0.70, ">"),
    "precision": ("Precision", 0.75, ">"),
    "recall": ("Recall (Sensibilidad)", 0.65, ">"),
    "f1_score": ("F1-Score", 0.70, ">"),
    "auc_roc": ("AUC-ROC", 0.70, ">"),
    "costo_asimetrico": ("Costo Asimétrico", 0.25, "<"),
}

print(f"{'='*65}")
print(f"{'Métrica':<25} {'Valor':>8} {'Objetivo':>10} {'Estado':>10}")
print(f"{'-'*65}")

cumplidos = 0
for key, (nombre, obj, comp) in objetivos.items():
    valor = metricas[key]
    if comp == ">":
        ok = valor > obj
        obj_str = f"> {obj:.2f}"
    else:
        ok = valor < obj
        obj_str = f"< {obj:.2f}"
    
    estado = "✓ CUMPLE" if ok else "✗ NO CUMPLE"
    cumplidos += int(ok)
    print(f"  {nombre:<23} {valor:>8.4f} {obj_str:>10} {estado:>12}")

print(f"{'-'*65}")
print(f"  Resultado: {cumplidos}/6 objetivos cumplidos")
print(f"{'='*65}")

# Métricas adicionales — la clave en calculate_all() es "matriz"
cm = metricas["matriz"]
print(f"\nMatriz de Confusión:")
print(f"  ┌─────────────────┬──────────┬──────────┐")
print(f"  │                 │ Pred = 1 │ Pred = 0 │")
print(f"  │                 │(Aprueba) │(Rechaza) │")
print(f"  ├─────────────────┼──────────┼──────────┤")
print(f"  │ Real = 1 (Bueno)│ VP = {cm['VP']:>3} │ FN = {cm['FN']:>3} │")
print(f"  │ Real = 0 (Malo) │ FP = {cm['FP']:>3} │ VN = {cm['VN']:>3} │")
print(f"  └─────────────────┴──────────┴──────────┘")
print(f"\n  Total: {cm['total']} registros")
print(f"  VP (Verdaderos Positivos): {cm['VP']} — Buenos pagadores correctamente aprobados")
print(f"  FP (Falsos Positivos):     {cm['FP']} — {cm['descripcion_FP']}")
print(f"  VN (Verdaderos Negativos): {cm['VN']} — Malos pagadores correctamente rechazados")
print(f"  FN (Falsos Negativos):     {cm['FN']} — {cm['descripcion_FN']}")

Métrica                      Valor   Objetivo     Estado
-----------------------------------------------------------------
  Accuracy                  0.4590     > 0.70  ✗ NO CUMPLE
  Precision                 0.7416     > 0.75  ✗ NO CUMPLE
  Recall (Sensibilidad)     0.3486     > 0.65  ✗ NO CUMPLE
  F1-Score                  0.4742     > 0.70  ✗ NO CUMPLE
  AUC-ROC                   0.5512     > 0.70  ✗ NO CUMPLE
  Costo Asimétrico          0.7960     < 0.25  ✗ NO CUMPLE
-----------------------------------------------------------------
  Resultado: 0/6 objetivos cumplidos

Matriz de Confusión:
  ┌─────────────────┬──────────┬──────────┐
  │                 │ Pred = 1 │ Pred = 0 │
  │                 │(Aprueba) │(Rechaza) │
  ├─────────────────┼──────────┼──────────┤
  │ Real = 1 (Bueno)│ VP = 244 │ FN = 456 │
  │ Real = 0 (Malo) │ FP =  85 │ VN = 215 │
  └─────────────────┴──────────┴──────────┘

  Total: 1000 registros
  VP (Verdaderos Positivos): 244 — Buenos pagadores correctamente

---
## 5. Visualizaciones

Se generan **5 gráficos** para documentar el desempeño del motor:

1. **Matriz de Confusión** — Distribución de VP, FP, VN, FN  
2. **Curva ROC** — Capacidad discriminatoria vs línea aleatoria  
3. **Distribución de Scores** — Separación entre buenos y malos pagadores  
4. **Curva Precision-Recall** — Trade-off entre precisión y exhaustividad  
5. **Dashboard Integral** — Resumen visual de todas las métricas

In [8]:
# ═══════════════════════════════════════════════════════════
# 5.1 MATRIZ DE CONFUSIÓN
# ═══════════════════════════════════════════════════════════

plots = MIHACPlots()
plots.plot_confusion_matrix(metricas)
plt.show()

In [9]:
# ═══════════════════════════════════════════════════════════
# 5.2 CURVA ROC
# ═══════════════════════════════════════════════════════════

y_real = bt.results_df["y_real"].values
scores = bt.results_df["score_mihac"].values

plots.plot_roc_curve(y_real, scores)
plt.show()

In [10]:
# ═══════════════════════════════════════════════════════════
# 5.3 DISTRIBUCIÓN DE SCORES
# ═══════════════════════════════════════════════════════════

plots.plot_score_distribution(scores, y_real)
plt.show()

In [11]:
# ═══════════════════════════════════════════════════════════
# 5.4 CURVA PRECISION-RECALL
# ═══════════════════════════════════════════════════════════

plots.plot_precision_recall_curve(y_real, scores)
plt.show()

In [12]:
# ═══════════════════════════════════════════════════════════
# 5.5 DASHBOARD INTEGRAL
# ═══════════════════════════════════════════════════════════

plots.plot_metrics_dashboard(metricas, y_real, scores)
plt.show()

---
## 6. Análisis de Errores (FP / FN)

Los errores se desagregan en dos categorías críticas:

- **Falsos Positivos (FP):** El motor aprobó a clientes que resultaron malos pagadores.  
  Esto representa **pérdida directa de capital** y es el error más costoso (peso = 4).

- **Falsos Negativos (FN):** El motor rechazó a clientes que eran buenos pagadores.  
  Esto representa **costo de oportunidad** (peso = 1).

Se analizan los perfiles típicos de cada tipo de error para identificar las  
variables y rangos que causan mayor confusión al motor.

In [13]:
# ═══════════════════════════════════════════════════════════
# 6.1 PERFIL DE ERRORES
# ═══════════════════════════════════════════════════════════

df_results = bt.results_df.copy()

# Clasificar cada registro
df_results["tipo_resultado"] = "—"
df_results.loc[(df_results["y_pred"] == 1) & (df_results["y_real"] == 1), "tipo_resultado"] = "VP"
df_results.loc[(df_results["y_pred"] == 1) & (df_results["y_real"] == 0), "tipo_resultado"] = "FP"
df_results.loc[(df_results["y_pred"] == 0) & (df_results["y_real"] == 0), "tipo_resultado"] = "VN"
df_results.loc[(df_results["y_pred"] == 0) & (df_results["y_real"] == 1), "tipo_resultado"] = "FN"

# Separar errores
df_fp = df_results[df_results["tipo_resultado"] == "FP"]
df_fn = df_results[df_results["tipo_resultado"] == "FN"]
df_vp = df_results[df_results["tipo_resultado"] == "VP"]
df_vn = df_results[df_results["tipo_resultado"] == "VN"]

# Tabla comparativa de perfiles
vars_comp = ["edad", "ingreso_mensual", "total_deuda_actual", "dti_mihac", "score_mihac"]

perfil_data = []
for nombre, grupo in [("VP", df_vp), ("FP", df_fp), ("VN", df_vn), ("FN", df_fn)]:
    row = {"Grupo": nombre, "N": len(grupo)}
    for v in vars_comp:
        if len(grupo) > 0:
            row[v] = f"{grupo[v].mean():.2f}"
        else:
            row[v] = "—"
    perfil_data.append(row)

perfil_df = pd.DataFrame(perfil_data)
print("Perfil promedio por tipo de resultado:\n")
print(perfil_df.to_string(index=False))

# Propósito más común en FP
if len(df_fp) > 0:
    print(f"\nFP — Propósito más común: {df_fp['proposito_credito'].mode().iloc[0]}")
    print(f"FP — Distribución por propósito:")
    for prop, cnt in df_fp["proposito_credito"].value_counts().items():
        print(f"  {prop}: {cnt} ({cnt/len(df_fp)*100:.1f}%)")

# Dictamen original de FN
if len(df_fn) > 0:
    print(f"\nFN — Distribución por dictamen original:")
    for d, cnt in df_fn["dictamen"].value_counts().items():
        print(f"  {d}: {cnt} ({cnt/len(df_fn)*100:.1f}%)")

Perfil promedio por tipo de resultado:

Grupo   N  edad ingreso_mensual total_deuda_actual dti_mihac score_mihac
   VP 244 35.93        18189.08            4954.04      0.27       96.94
   FP  85 35.75        33259.77            9698.11      0.31       94.53
   VN 215 33.26        13825.62            5123.85      0.34       40.71
   FN 456 36.38        18409.76            5623.09      0.30       42.22

FP — Propósito más común: Consumo
FP — Distribución por propósito:
  Consumo: 63 (74.1%)
  Negocio: 14 (16.5%)
  Educacion: 7 (8.2%)
  Emergencia: 1 (1.2%)

FN — Distribución por dictamen original:
  RECHAZADO: 375 (82.2%)
  REVISION_MANUAL: 81 (17.8%)


In [14]:
# ═══════════════════════════════════════════════════════════
# 6.2 COMPARACIÓN VISUAL: SCORES POR TIPO DE RESULTADO
# ═══════════════════════════════════════════════════════════

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot de scores por tipo
color_map = {"VP": VERDE, "FP": ROJO, "VN": AZUL, "FN": AMBAR}
orden = ["VP", "FP", "VN", "FN"]

grupos_data = [df_results[df_results["tipo_resultado"] == g]["score_mihac"].values for g in orden]
bp = axes[0].boxplot(grupos_data, labels=orden, patch_artist=True, widths=0.6)
for patch, color in zip(bp["boxes"], [color_map[g] for g in orden]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
axes[0].set_title("Distribución de Scores por Tipo de Resultado", fontweight="bold")
axes[0].set_ylabel("Score MIHAC (0-100)")
axes[0].axhline(y=80, color=GRIS, linestyle="--", alpha=0.7, label="Umbral = 80")
axes[0].legend()

# Ingreso mensual por tipo
grupos_ingreso = [df_results[df_results["tipo_resultado"] == g]["ingreso_mensual"].values for g in orden]
bp2 = axes[1].boxplot(grupos_ingreso, labels=orden, patch_artist=True, widths=0.6)
for patch, color in zip(bp2["boxes"], [color_map[g] for g in orden]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
axes[1].set_title("Ingreso Mensual por Tipo de Resultado", fontweight="bold")
axes[1].set_ylabel("Ingreso Mensual ($)")

plt.tight_layout()
plt.show()

In [15]:
# ═══════════════════════════════════════════════════════════
# 6.3 VARIABLES CRÍTICAS — Heatmap de diferencias
# ═══════════════════════════════════════════════════════════

# Calcular promedios normalizados por grupo
vars_analisis = ["edad", "ingreso_mensual", "total_deuda_actual", 
                 "antiguedad_laboral", "monto_credito", "numero_dependientes"]

heat_data = {}
for grupo_name, grupo_df in [("VP", df_vp), ("FP", df_fp), ("VN", df_vn), ("FN", df_fn)]:
    if len(grupo_df) > 0:
        heat_data[grupo_name] = {v: grupo_df[v].mean() for v in vars_analisis}
    else:
        heat_data[grupo_name] = {v: 0 for v in vars_analisis}

heat_df = pd.DataFrame(heat_data).T

# Normalizar por columna para comparabilidad
heat_norm = (heat_df - heat_df.min()) / (heat_df.max() - heat_df.min() + 1e-10)

fig, ax = plt.subplots(figsize=(12, 4))
sns.heatmap(heat_norm, annot=heat_df.round(1).values, fmt="", cmap="RdYlGn_r",
            linewidths=1, ax=ax, cbar_kws={"label": "Valor normalizado"})
ax.set_title("Perfil Promedio por Tipo de Resultado (valores originales en celdas)", 
             fontweight="bold", fontsize=13)
ax.set_ylabel("Tipo de Resultado")
ax.set_xticklabels([v.replace("_", "\n") for v in vars_analisis], rotation=0)
plt.tight_layout()
plt.show()

# Tabla de diferencias porcentuales FP vs VP
print("\nDiferencia porcentual FP vs VP (variables numéricas):")
print(f"{'Variable':<25} {'VP (prom)':>12} {'FP (prom)':>12} {'Diff %':>10}")
print("-" * 60)
for v in vars_analisis:
    vp_val = df_vp[v].mean() if len(df_vp) > 0 else 0
    fp_val = df_fp[v].mean() if len(df_fp) > 0 else 0
    diff_pct = ((fp_val - vp_val) / (vp_val + 1e-10)) * 100
    print(f"  {v:<23} {vp_val:>12.2f} {fp_val:>12.2f} {diff_pct:>+9.1f}%")


Diferencia porcentual FP vs VP (variables numéricas):
Variable                     VP (prom)    FP (prom)     Diff %
------------------------------------------------------------
  edad                           35.93        35.75      -0.5%
  ingreso_mensual             18189.08     33259.77     +82.9%
  total_deuda_actual           4954.04      9698.11     +95.8%
  antiguedad_laboral              4.50         4.25      -5.5%
  monto_credito               25156.50     32012.39     +27.3%
  numero_dependientes             1.00         1.00      +0.0%


---
## 7. Calibración de Pesos

El módulo `WeightCalibrator` analiza los errores del backtesting y propone ajustes  
heurísticos a los pesos y umbrales del motor. 

**Importante — Esto NO es Machine Learning:**  
La calibración **no** usa gradient descent ni optimización numérica automatizada.  
Analiza patrones estadísticos en los errores y propone ajustes conservadores que  
un experto humano debe revisar.

**Principio de conservadurismo:**
- Cambios máximos de pesos: ±2% por iteración  
- Cambio máximo de umbral: ±3 puntos  
- Prioridad: reducir FP sobre reducir FN (ratio 4:1)

In [16]:
# ═══════════════════════════════════════════════════════════
# 7.1 ANÁLISIS DE ERRORES CON CALIBRADOR
# ═══════════════════════════════════════════════════════════

cal = WeightCalibrator()
analisis = cal.analyze_errors(bt.results_df)

# Variables críticas
print(f"{'='*65}")
print("Variables Críticas (por impacto en error)")
print(f"{'='*65}")
print(f"{'Variable':<25} {'Criticidad':>12} {'FP diff %':>10} {'FN diff %':>10}")
print("-" * 65)

for vc in analisis.get("variables_criticas", []):
    print(f"  {vc['variable']:<23} {vc['criticidad']:>12.2f} {vc['diff_fp_pct']:>9.0f}% {vc['diff_fn_pct']:>9.0f}%")

# Análisis de umbrales
print(f"\n{'='*65}")
print("Análisis de Sensibilidad por Umbral")
print(f"{'='*65}")

umbral_info = analisis["umbrales_analisis"]
print(f"Umbral actual: {umbral_info.get('umbral_actual', 80)}")
print(f"Umbral óptimo (costo): {umbral_info['umbral_optimo_costo']}")

print(f"\n{'Umbral':>7} | {'Acc':>6} | {'Prec':>6} | {'Rec':>6} | {'F1':>6} | {'Costo':>6} | {'FP':>4} | {'FN':>4}")
print("-" * 65)
for u in sorted(umbral_info["por_umbral"].keys()):
    r = umbral_info["por_umbral"][u]
    marker = " ◄" if u == 80 else ""
    print(f"  {u:>5} | {r['accuracy']:.3f} | {r['precision']:.3f} | {r['recall']:.3f} | "
          f"{r['f1_score']:.3f} | {r['costo_asimetrico']:.3f} | {r['fp']:>3} | {r['fn']:>3}{marker}")

Variables Críticas (por impacto en error)
Variable                    Criticidad  FP diff %  FN diff %
-----------------------------------------------------------------
  ingreso_mensual                 5.63       141%         1%
  total_deuda_actual              3.71        89%        14%
  dti_mihac                       0.49        10%        10%
  edad                            0.31         8%         1%

Análisis de Sensibilidad por Umbral
Umbral actual: 80
Umbral óptimo (costo): 90

 Umbral |    Acc |   Prec |    Rec |     F1 |  Costo |   FP |   FN
-----------------------------------------------------------------
     70 | 0.491 | 0.729 | 0.434 | 0.544 | 0.848 | 113 | 396
     75 | 0.472 | 0.724 | 0.397 | 0.513 | 0.846 | 106 | 422
     78 | 0.469 | 0.733 | 0.380 | 0.500 | 0.822 |  97 | 434
     80 | 0.463 | 0.736 | 0.363 | 0.486 | 0.810 |  91 | 446 ◄
     82 | 0.460 | 0.742 | 0.350 | 0.476 | 0.795 |  85 | 455
     85 | 0.454 | 0.744 | 0.336 | 0.463 | 0.789 |  81 | 465
     88 | 

In [17]:
# ═══════════════════════════════════════════════════════════
# 7.2 VISUALIZACIÓN: Sensibilidad al Umbral
# ═══════════════════════════════════════════════════════════

umbrales = sorted(umbral_info["por_umbral"].keys())
accs = [umbral_info["por_umbral"][u]["accuracy"] for u in umbrales]
precs = [umbral_info["por_umbral"][u]["precision"] for u in umbrales]
recs = [umbral_info["por_umbral"][u]["recall"] for u in umbrales]
f1s = [umbral_info["por_umbral"][u]["f1_score"] for u in umbrales]
costos = [umbral_info["por_umbral"][u]["costo_asimetrico"] for u in umbrales]
fps = [umbral_info["por_umbral"][u]["fp"] for u in umbrales]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Métricas vs Umbral
axes[0].plot(umbrales, precs, 'o-', color=AZUL, label="Precision", linewidth=2)
axes[0].plot(umbrales, recs, 's-', color=VERDE, label="Recall", linewidth=2)
axes[0].plot(umbrales, f1s, '^-', color=AMBAR, label="F1-Score", linewidth=2)
axes[0].axvline(x=80, color=GRIS, linestyle="--", alpha=0.7, label="Umbral actual (80)")
axes[0].set_xlabel("Umbral de Aprobación")
axes[0].set_ylabel("Valor de la Métrica")
axes[0].set_title("Métricas vs Umbral de Aprobación", fontweight="bold")
axes[0].legend(loc="best")
axes[0].set_ylim(0, 1)
axes[0].grid(True, alpha=0.3)

# Costo y FP vs Umbral
ax2 = axes[1]
color1 = ROJO
ax2.plot(umbrales, costos, 'D-', color=color1, linewidth=2, label="Costo Asimétrico")
ax2.set_xlabel("Umbral de Aprobación")
ax2.set_ylabel("Costo Asimétrico", color=color1)
ax2.tick_params(axis='y', labelcolor=color1)
ax2.axvline(x=80, color=GRIS, linestyle="--", alpha=0.7)

ax3 = ax2.twinx()
ax3.bar(umbrales, fps, alpha=0.3, color=AZUL, width=1.5, label="FP")
ax3.set_ylabel("Falsos Positivos", color=AZUL)
ax3.tick_params(axis='y', labelcolor=AZUL)

ax2.set_title("Costo Asimétrico y FP vs Umbral", fontweight="bold")
lines1, labels1 = ax2.get_legend_handles_labels()
lines2, labels2 = ax3.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc="upper right")

plt.tight_layout()
plt.show()

In [18]:
# ═══════════════════════════════════════════════════════════
# 7.3 PROPUESTAS DE AJUSTE
# ═══════════════════════════════════════════════════════════

propuestas = cal.propose_adjustments(analisis)

print(f"{'='*65}")
print("Propuestas de Ajuste de Pesos")
print(f"{'='*65}")

if propuestas["cambios_pesos"]:
    print(f"\n{'Variable':<25} {'Antes':>8} {'Después':>8} {'Delta':>8} {'Razón'}")
    print("-" * 75)
    for c in propuestas["cambios_pesos"]:
        print(f"  {c['variable']:<23} {c['peso_anterior']:>8.2f} {c['peso_nuevo']:>8.2f} "
              f"{c['peso_nuevo']-c['peso_anterior']:>+7.2f}   {c['razon']}")
else:
    print("\nNo se proponen cambios de pesos.")

print(f"\nUmbral: {propuestas['umbral_actual']} → {propuestas['umbral_propuesto']}")
print(f"Justificación: {propuestas.get('umbral_justificacion', 'Ajuste conservador basado en costo asimétrico')}")

Propuestas de Ajuste de Pesos

Variable                     Antes  Después    Delta Razón
---------------------------------------------------------------------------
  Ingreso_Mensual             0.32     0.30   -0.02   Ingreso_Mensual: FP difieren 141% de VN → reducir peso de 0.32 a 0.30
  Total_Deuda_Actual          0.13     0.11   -0.02   Total_Deuda_Actual: FP difieren 89% de VN → reducir peso de 0.13 a 0.11

Umbral: 80 → 83
Justificación: Ajuste conservador basado en costo asimétrico


---
## 8. Comparación Antes vs Después (Simulación)

Se simula el impacto de los ajustes propuestos re-evaluando los 1,000 registros  
con los nuevos pesos y umbral. Esto permite comparar las métricas **antes** y  
**después** sin modificar permanentemente el motor.

In [19]:
# ═══════════════════════════════════════════════════════════
# 8. SIMULACIÓN: ANTES vs DESPUÉS
# ═══════════════════════════════════════════════════════════

# Re-cargar datos para simulación
mapper2 = GermanCreditMapper()
df2 = mapper2.load_and_transform(None)
datos_mihac = mapper2.to_mihac_dicts(df2)
y_real_sim = df2["etiqueta_binaria"].values

comparacion = cal.simulate(propuestas, datos_mihac, y_real_sim, verbose=True)


── Simulando impacto de propuestas ──
  Evaluando 1000 registros...
  ✓ Evaluados en 1.8s

  Umbral: 80 → 83
  accuracy            : 0.4590 → 0.4600 (+0.0010) ↑
  precision           : 0.7416 → 0.7454 (+0.0038) ↑
  recall              : 0.3486 → 0.3471 (-0.0015) ↓
  f1_score            : 0.4742 → 0.4737 (-0.0005) ↓
  specificity         : 0.7167 → 0.7233 (+0.0066) ↑
  auc_roc             : 0.5512 → 0.5512 (0.0000) =
  costo_asimetrico    : 0.7960 → 0.7890 (-0.0070) ↓

  Recomendación: APLICAR


In [21]:
# ═══════════════════════════════════════════════════════════
# 8.2 TABLA COMPARATIVA Y VISUALIZACIÓN
# ═══════════════════════════════════════════════════════════

m_antes = comparacion["metricas_antes"]
m_despues = comparacion["metricas_despues"]

metricas_keys = ["accuracy", "precision", "recall", "f1_score", "specificity", "auc_roc", "costo_asimetrico"]
nombres_metricas = ["Accuracy", "Precision", "Recall", "F1-Score", "Specificity", "AUC-ROC", "Costo Asimétrico"]

print(f"{'='*65}")
print("Comparación: Antes (v1) vs Después (v2)")
print(f"{'='*65}")
print(f"{'Métrica':<22} | {'Antes':>8} | {'Después':>8} | {'Delta':>10}")
print("-" * 65)

for key, nombre in zip(metricas_keys, nombres_metricas):
    antes_val = m_antes[key]
    despues_val = m_despues[key]
    delta = despues_val - antes_val
    signo = "+" if delta > 0 else ""
    print(f"  {nombre:<20} | {antes_val:>8.4f} | {despues_val:>8.4f} | {signo}{delta:>9.4f}")

# Matriz antes vs después
cm_a = m_antes["matriz"]
cm_d = m_despues["matriz"]
print(f"\nMatriz de Confusión:")
print(f"  ANTES:   VP={cm_a['VP']}  FP={cm_a['FP']}  VN={cm_a['VN']}  FN={cm_a['FN']}")
print(f"  DESPUÉS: VP={cm_d['VP']}  FP={cm_d['FP']}  VN={cm_d['VN']}  FN={cm_d['FN']}")

# Gráfico comparativo
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(metricas_keys))
width = 0.35

vals_antes = [m_antes[k] for k in metricas_keys]
vals_despues = [m_despues[k] for k in metricas_keys]

bars1 = ax.bar(x - width/2, vals_antes, width, label="Antes (v1)", color=AZUL, alpha=0.8)
bars2 = ax.bar(x + width/2, vals_despues, width, label="Después (v2)", color=VERDE, alpha=0.8)

ax.set_xticks(x)
ax.set_xticklabels(nombres_metricas, rotation=30, ha="right")
ax.set_ylabel("Valor")
ax.set_title("Comparación de Métricas: v1 vs v2 (Post-Calibración)", fontweight="bold", fontsize=13)
ax.legend()
ax.set_ylim(0, 1.05)
ax.grid(axis="y", alpha=0.3)

# Agregar valores sobre barras
for bar in bars1:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f"{bar.get_height():.3f}", ha="center", va="bottom", fontsize=8)
for bar in bars2:
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f"{bar.get_height():.3f}", ha="center", va="bottom", fontsize=8)

plt.tight_layout()
plt.show()

Comparación: Antes (v1) vs Después (v2)
Métrica                |    Antes |  Después |      Delta
-----------------------------------------------------------------
  Accuracy             |   0.4590 |   0.4600 | +   0.0010
  Precision            |   0.7416 |   0.7454 | +   0.0038
  Recall               |   0.3486 |   0.3471 |   -0.0015
  F1-Score             |   0.4742 |   0.4737 |   -0.0005
  Specificity          |   0.7167 |   0.7233 | +   0.0066
  AUC-ROC              |   0.5512 |   0.5512 |    0.0000
  Costo Asimétrico     |   0.7960 |   0.7890 |   -0.0070

Matriz de Confusión:
  ANTES:   VP=244  FP=85  VN=215  FN=456
  DESPUÉS: VP=243  FP=83  VN=217  FN=457


In [22]:
# ═══════════════════════════════════════════════════════════
# 8.3 REPORTE TEXTUAL DE CALIBRACIÓN
# ═══════════════════════════════════════════════════════════

reporte_cal = cal.generate_report(analisis, propuestas, comparacion)
print(reporte_cal)

REPORTE DE CALIBRACIÓN — MIHAC v1.0

── ANÁLISIS DE ERRORES ──
Total registros: 1000
  FP: 85
  FN: 456
  VP: 244
  VN: 215

── VARIABLES CRÍTICAS (por impacto en error) ──
  ingreso_mensual           Criticidad=5.63  FP diff=141%  FN diff=1%
  total_deuda_actual        Criticidad=3.71  FP diff=89%  FN diff=14%
  dti_mihac                 Criticidad=0.49  FP diff=10%  FN diff=10%
  edad                      Criticidad=0.31  FP diff=8%  FN diff=1%

── ANÁLISIS DE UMBRALES ──
  Umbral actual: 80
  Umbral óptimo (costo): 90

   Umbral |    Acc |   Prec |    Rec |     F1 |  Costo |   FP |   FN
  --------------------------------------------------------
       70 |  0.491 |  0.729 |  0.434 |  0.544 |  0.848 |  113 |  396
       75 |  0.472 |  0.724 |  0.397 |  0.513 |  0.846 |  106 |  422
       78 |  0.469 |  0.733 |  0.380 |  0.500 |  0.822 |   97 |  434
       80 |  0.463 |  0.736 |  0.363 |  0.486 |  0.810 |   91 |  446
       82 |  0.460 |  0.742 |  0.350 |  0.476 |  0.795 |   85 |  455

---
## 9. Discusión y Limitaciones

### 9.1 Interpretación de Resultados

Los resultados del backtesting muestran que el motor MIHAC, como sistema experto  
basado en reglas heurísticas, presenta **Precision aceptable (≈0.74)** pero  
**Recall bajo (≈0.35)**, lo que indica un comportamiento **conservador**: cuando  
aprueba, generalmente acierta (74% de las veces), pero deja fuera a muchos  
buenos pagadores (solo identifica al 35% de ellos).

**Hallazgos clave:**

1. **El motor es conservador por diseño.** La combinación de umbral alto (80),  
   regla de DTI CRITICO (rechazo inmediato ≥0.60), y penalizaciones acumulativas  
   de las 15 reglas genera un sesgo hacia el rechazo. Esto es coherente con el  
   principio financiero de preferir rechazar un buen cliente (FN) antes que  
   aprobar un mal cliente (FP).

2. **La Precision se mantiene estable** (0.74) a través de diferentes umbrales,  
   indicando que la estructura de sub-scores discrimina razonablemente entre  
   los registros que reciben scores altos.

3. **El Recall es el punto débil.** La mayoría de los buenos pagadores (456/700 = 65%)  
   son rechazados. Esto se debe en parte a la naturaleza del mapeo de variables  
   (ver limitaciones) y en parte al diseño conservador del motor.

4. **El costo asimétrico (≈0.80)** es alto, pero esto se explica por el volumen  
   de FN (456) más que por el volumen de FP (85). En un escenario real, los  
   FN representan oportunidad perdida (costo = 1 cada uno), mientras que los  
   FP representan pérdida directa (costo = 4 cada uno).

### 9.2 Limitaciones del Benchmark

Es fundamental contextualizar estos resultados con las **limitaciones inherentes**  
del German Credit Dataset como benchmark para MIHAC:

1. **Variables proxy vs variables reales.** El mapper transforma las 20 variables  
   originales del dataset (codificadas en formato categórico alemán) a las 9  
   variables de entrada MIHAC. Esta transformación es necesariamente una  
   **aproximación**: por ejemplo, `ingreso_mensual` se estima a partir de  
   `existing_credits`, `employment`, `foreign_worker` y `job`, no de un  
   ingreso real declarado. Esto introduce ruido irreducible.

2. **Dataset de 1994.** Los patrones de riesgo crediticio de la Alemania  
   reunificada de 1994 difieren significativamente de los patrones  
   contemporáneos de crédito en mercados latinoamericanos, que son el  
   contexto objetivo de MIHAC.

3. **Etiquetas binarias simplificadas.** El dataset classifica cada crédito  
   como "bueno" (1) o "malo" (2), sin matices como grado de morosidad,  
   reestructuración, o pago anticipado.

4. **Sin variables dinámicas.** MIHAC está diseñado para operar con información  
   actualizada en tiempo real (ingresos verificados, deuda consultada en buró).  
   El dataset German Credit proporciona una instantánea estática sin  
   actualización posterior.

5. **Tamaño limitado (n=1,000).** Con solo 300 malos pagadores y 700 buenos,  
   el espacio de validación es acotado. Un dataset de producción tendría  
   órdenes de magnitud más registros.

### 9.3 Sistema Experto vs Machine Learning

MIHAC es un **sistema experto basado en reglas**, no un modelo de Machine Learning.  
Esta distinción es fundamental para interpretar las métricas:

| Aspecto | Sistema Experto (MIHAC) | ML (Random Forest, XGBoost) |
|---------|------------------------|---------------------------|
| Transparencia | ✓ Cada decisión es explicable | Requiere técnicas SHAP/LIME |
| Reglas explícitas | ✓ 15 reglas IF-THEN documentadas | Patrones aprendidos implícitamente |
| Rendimiento predictivo | Limitado por diseño manual | Generalmente superior |
| Dependencia de datos | Baja (opera con conocimiento experto) | Alta (requiere datos de entrenamiento) |
| Mantenibilidad | ✓ Modificable por analistas | Requiere re-entrenamiento |
| Regulación | ✓ Cumple requisitos de explicabilidad | Puede ser "caja negra" |

La comparación directa de métricas (AUC, F1) con modelos ML no es apropiada  
porque MIHAC no fue diseñado para maximizar estas métricas, sino para  
**codificar políticas de crédito explicables y auditables**.

### 9.4 Valor del Sistema

A pesar de las métricas subóptimas en el benchmark, MIHAC aporta valor en:

- **Explicabilidad:** Cada dictamen incluye las reglas activadas, sub-scores  
  y compensaciones que justifican la decisión.
- **Auditabilidad:** El flujo de decisión es completamente rastreable y  
  documentable para reguladores.
- **Ajustabilidad:** Los pesos, umbrales y reglas pueden modificarse por  
  analistas sin re-entrenamiento de modelos.
- **Velocidad:** >1,000 evaluaciones/segundo, adecuado para producción.

---
## 10. Conclusiones

### Resumen de la Validación Empírica

Se completó el pipeline de validación empírica del motor MIHAC con los siguientes  
entregables:

| # | Entregable | Estado |
|---|-----------|--------|
| 1 | `validation/metrics.py` — Módulo de métricas (8/8 tests) | ✓ Completo |
| 2 | `validation/backtesting.py` — Módulo de backtesting (5/5 tests) | ✓ Completo |
| 3 | `validation/calibrator.py` — Módulo de calibración (6/6 tests) | ✓ Completo |
| 4 | Ejecución de calibración con pesos v2 (25/25 tests) | ✓ Completo |
| 5 | `notebooks/02_validation_report.ipynb` — Este notebook | ✓ Completo |

### Métricas Finales (Post-Calibración v2)

El proceso de calibración logró una **reducción marginal del costo asimétrico**  
manteniendo la estabilidad del sistema. Los cambios conservadores (±2% pesos,  
±3 umbral) son coherentes con el principio de no degradar el comportamiento  
existente.

### Trabajo Futuro

1. **Validación con datos reales:** Probar MIHAC con datos de una institución  
   financiera local para obtener métricas representativas del contexto objetivo.
2. **Calibración iterativa:** Ejecutar múltiples ciclos de calibración con  
   datos de producción para convergencia gradual.
3. **Reglas adicionales:** Incorporar reglas basadas en nuevas variables  
   (historial de pagos, consultas a buró, antigüedad de cuenta).
4. **Umbral dinámico:** Implementar umbrales diferenciados por segmento  
   de cliente (monto, propósito, perfil de riesgo).

---
*Notebook generado como parte de la validación empírica del motor MIHAC v1.0.*  
*Módulos utilizados: `metrics.py`, `backtesting.py`, `calibrator.py`*